In [20]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json

In [21]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [5]:
global oddschecker
oddschecker = 'https://www.oddschecker.com/football/'

In [15]:
def get_stats(url):
    driver = webdriver.Chrome()
    try:
        driver.implicitly_wait(random.randint(0, 6))
        driver.get(url)
        ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        if ht != 'Half-Time':
            print('not halftime just yet ... sleeping')
            time.sleep(3*60)
            ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
            if ht != 'Half-Time':
                print(f'time: {ht}')
        button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
        button.click()
        div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
        t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
              if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
        t['score'] = [score[0], score[2]]
        posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
        t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
        game_stats = {}
        for name, stats in t.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
#         game_stats = {k: float(v[0]) - float(v[1]) for k, v in game_stats.items()}
        game_stats['match_name'] = url.split('/')[-1]
        game_stats['half_time'] = ht
        game_stats['url'] = url
    except:
        game_stats = {}
    driver.quit()
    return game_stats

In [16]:
get_stats('https://www.fotmob.com/match/3900943/matchfacts/aston-villa-vs-everton')

not halftime just yet ... sleeping
time: Full-Time


{}

In [22]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership'}

def fotmob_game_data(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-aoqfcq-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    for count, bucket in enumerate(divs):
        league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
        if league_name not in xg_leagues.keys():
            continue
        url = bucket.find_elements(By.TAG_NAME, 'a')
        urls = [u.get_attribute('href') for u in url]
        time = bucket.text
        time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=53) for i in time.split('\n') if ':' in i]
        urls = urls[1:]
        names = [u.split('/')[-1] for u in urls]
        league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    return league

In [23]:
game_data = fotmob_game_data('https://www.fotmob.com/?date=20220824')

In [24]:
game_data

{'MEXICO - LIGA MX APERTURA': {'club-universidad-nacional-vs-tigres': ['https://www.fotmob.com/match/3888547/matchfacts/club-universidad-nacional-vs-tigres',
   datetime.datetime(2022, 8, 24, 22, 58)]}}

In [14]:
game_data

{'UNITED STATES - MAJOR LEAGUE SOCCER': {'los-angeles-fc-vs-dc-united': ['https://www.fotmob.com/match/3787569/matchfacts/los-angeles-fc-vs-dc-united',
   datetime.datetime(2022, 8, 16, 23, 23),
   'https://www.oddschecker.com/football/world/usa/mls/los-angeles-fc-v-dc-united/winner']},
 'ENGLAND - CHAMPIONSHIP': {'birmingham-city-vs-watford': ['https://www.fotmob.com/match/3915280/matchfacts/birmingham-city-vs-watford',
   datetime.datetime(2022, 8, 16, 15, 38),
   'https://www.oddschecker.com/football/english/championship/birmingham-v-watford/winner']},
 'MEXICO - LIGA MX APERTURA': {'atlas-vs-fc-juarez': ['https://www.fotmob.com/match/3888526/matchfacts/atlas-vs-fc-juarez',
   datetime.datetime(2022, 8, 16, 20, 53),
   'https://www.oddschecker.com/football/world/mexico/liga-mx/atlas-v-fc-juarez/winner'],
  'mazatlan-fc-vs-queretaro': ['https://www.fotmob.com/match/3888527/matchfacts/mazatlan-fc-vs-queretaro',
   datetime.datetime(2022, 8, 16, 20, 53),
   'https://www.oddschecker.com

In [20]:
def get_all_odds_links():
    odds_links = {}
    for f_league, url_league in xg_leagues.items():
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = oddschecker + url_league
        driver.get(url)
        driver.implicitly_wait(3)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt
    return odds_links

In [21]:
all_odds = get_all_odds_links()

In [22]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = collections.defaultdict(list)
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[str(l1_t_l2[1])].append([game_name, l_name, l1_t_l2[0], l1_t_l2[2]])
    scraper_data = dict(sorted(scraper_data.items(), key=lambda x: x[0]))
    return scraper_data

In [24]:
days_data = connect_links(game_data, all_odds)

In [78]:
days_data

{'2022-08-22 13:53:00': [['odense-vs-horsens',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887410/matchfacts/odense-vs-horsens',
   'https://www.oddschecker.com/football/denmark/superligaen/odense-bk-v-ac-horsens/winner']],
 '2022-08-22 14:53:00': [['elche-vs-almeria',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3917947/matchfacts/elche-vs-almeria',
   'https://www.oddschecker.com/football/spain/la-liga-primera/elche-v-almeria/winner']],
 '2022-08-22 15:53:00': [['manchester-united-vs-liverpool',
   'ENGLAND - PREMIER LEAGUE',
   'https://www.fotmob.com/match/3900958/matchfacts/manchester-united-vs-liverpool',
   'https://www.oddschecker.com/football/english/premier-league/man-utd-v-liverpool/winner']],
 '2022-08-22 16:08:00': [['gil-vicente-vs-famalicao',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937394/matchfacts/gil-vicente-vs-famalicao',
   'https://www.oddschecker.com/football/portugal/primeira-liga/gil-vicente-v-famalicao/winner'

In [14]:
def get_odds(url):
    try:
        driver = webdriver.Chrome()
        if url == '':
            return {'url':url, odds: ''}
        driver.implicitly_wait(1)
        url = url
        driver.get(url)
        odds = driver.find_element(By.XPATH, '//div[@class="ExpanderBestOddsWrapper_e1jc6hcf"]').text.split('\n')
        odds = [float(Fraction(j)) + 1.0 if '/' in j else j for j in odds]
        odds = dict(zip(odds[::2], odds[1::2]))
        odds = {'url':url, 'odds': odds}
    except:
        odds = {'url':url, 'odd': ''}
    driver.quit()
    return odds

In [15]:
odds = get_odds('https://www.oddschecker.com/football/english/premier-league/southampton-v-man-utd/winner')

In [16]:
odds

{'url': 'https://www.oddschecker.com/football/english/premier-league/southampton-v-man-utd/winner',
 'odds': {'Southampton': 4.5, 'Draw': 4.2, 'Man Utd': 1.8125}}

In [76]:
def stats_odds(stats_url, odds_url):
    stats = get_stats(stats_url)
    time.sleep(60)
    odds = get_odds(odds_url)
    row = {**stats, **odds}
    z.append(row)
    return row

In [29]:
c = days_data.copy()

In [96]:
while True:
    for clock in c.copy().keys():
        if datetime.strptime(clock, "%Y-%m-%d %H:%M:%S") <= datetime.now():
            stats_urls = [x[2] for x in c[clock]]
            odds_urls = [x[3] for x in c[clock]]
            print(stats_urls, odds_urls)
            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = list(executor.map(stats_odds, stats_urls, odds_urls))
            del c[clock]
        else:
            results = []
    print(f'{len(c)} halftimes left -- {len(z)}')
    if len(c) == 0:
        print('all done')
        break
    time.sleep(60)

5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes left -- 63
5 halftimes

0 halftimes left -- 68
all done


In [26]:
z

NameError: name 'z' is not defined

In [106]:
f = []
for dic in z:
    d = {}
    for k, v in dic.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    f.append(d)


In [110]:
df = pd.DataFrame(f).iloc[:,:45]
df = df[df['half_time'] == 'Half-Time']
df.to_csv('current_stats.csv')

In [25]:
df

NameError: name 'df' is not defined

In [102]:
z

[{'Expected goals (xG)': 0.37999999999999995,
  'Total shots': 2.0,
  'Big chances': 1.0,
  'Big chances missed': 1.0,
  'Accurate passes': 70.0,
  'Accurate passes percentage': 0.05999999999999994,
  'Fouls committed': 3.0,
  'Offsides': -1.0,
  'Corners': -1.0,
  'Shots off target': 2.0,
  'Shots on target': 2.0,
  'Blocked shots': -2.0,
  'Hit woodwork': 0.0,
  'Shots inside box': 2.0,
  'Shots outside box': 0.0,
  'xG open play': 0.43000000000000005,
  'xG set play': -0.05,
  'xG on target (xGOT)': 0.17,
  'Passes': 71.0,
  'Own half': 28.0,
  'Opposition half': 42.0,
  'Accurate long balls': 11.0,
  'Accurate long balls percentage': 0.26,
  'Accurate crosses': 4.0,
  'Accurate crosses percentage': 0.44,
  'Throws': -2.0,
  'Yellow cards': 0.0,
  'Red cards': 0.0,
  'Tackles won': 1.0,
  'Tackles won percentage': 0.0,
  'Interceptions': -3.0,
  'Blocks': 2.0,
  'Clearances': 5.0,
  'Keeper saves': -2.0,
  'Duels won': -3.0,
  'Ground duels won': -1.0,
  'Ground duels won percentage

In [101]:
# open_file = open('data.pkl', "wb")
# pickle.dump(z, open_file)
# open_file.close()

In [99]:
open_file = open('data.pkl', "rb")
loaded_data = pickle.load(open_file)
open_file.close()

In [18]:
open_file = open('20220823_data.pkl', "rb")
xcxc = pickle.load(open_file)
open_file.close()

In [19]:
xcxc

[]

In [100]:
len(loaded_data)

68

In [82]:
z = loaded_data